# Example
In this notebook we will see an example of how an explanation is generated and evaluated from start to fonosh

In [1]:
import dice_ml
from dice_ml.utils import helpers
import pandas as pd
import pickle
from prompts import *
from utils import *
from exp_machines import *

In [2]:
# Load the model we want to explain
with open("""./models/loan_model.pkl""", 'rb') as file:
    model = pickle.load(file)

#Load the train and test data set
train_dataset = pd.read_csv('./data/adult_train_dataset.csv')
test_dataset = pd.read_csv('./data/adult_test_dataset.csv')

#Load the examples we will try to explain
test_df = pd.read_csv('./data/test_examples.csv')
test_df = test_df.drop(columns=['income'], axis = 1)

In [3]:
exp_m = LLMExplanation4CFs(model = model, #Load the model we want to explain
                            model_description = """ML-system that predicts wether a person will earn more than 50k $ a year""", # brief explanation of the ML model
                            backend='sklearn', # Framework used to build the model (used to generate counterfactuals)
                            dataset_info=string_info(train_dataset.columns, helpers.get_adult_data_info()) , # string information about the dataset
                            continuous_features=['age', 'hours_per_week'], # Necessary for the counterfactual generation
                            outcome_name= 'income', #Necessary for counterfactual generation
                            training_set=train_dataset, #Necessary for counterfactual generation
                            test_set= test_dataset, #Necessary to  check novelty of the evaluation example
                            llm='gpt-4o', #LLM used, works with Langchain
                            prompt_type='zero', # zero or one
                            n_counterfactuals=5, #Number of counterfactuals used in the explanation 
                            user_input=False #Human in the loop helping select the causes
                           )


exp_m.fit()
counterfactuals, rules, code1, result1, explanation, code2, final_cf, code3, prediction, n_rules,rules_followed, first_rule, second_rule,third_rule, is_in_cfs, is_in_data = exp_m.explain_evaluate(example = test_df.iloc[[0]], verbose = False,return_all=True)

100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


We will looked at the following example of a woman who is predicted to earn less than 50k$ a year. We will look at the whole process followed by the LLM in order to obtain this final explanation.

In [4]:
print(test_df.iloc[0])

age                        29
workclass             Private
education             HS-grad
marital_status        Married
occupation        Blue-Collar
race                    White
gender                 Female
hours_per_week             38
Name: 0, dtype: object


First, a set of counterfactuals will be generated using the DiCE ML package

In [5]:
counterfactuals

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,Prof-school,Married,White-Collar,White,Female,38,1
1,29,Private,Some-college,Married,Professional,White,Female,38,1
2,29,Private,Doctorate,Married,Blue-Collar,White,Male,38,1
3,29,Private,Bachelors,Married,Other/Unknown,White,Female,38,1
4,29,Private,Bachelors,Married,Blue-Collar,White,Male,38,1


A set of rules is extracted from this counterfactual using the LLM

In [6]:
print(rules)

1. **Education Level**:
2. **Occupation**:
3. **Gender**:
4. **Work Hours**:
5. **Marital Status**:
6. **Race**:



In order to check whether this rules are correct or not, we ask the LLM to create a program that checks it. In the following cell we can see the code generated by the LLM.

In [7]:
print(code1)

import pandas as pd

# Given positive counterfactuals
counterfactuals_data = {
    'age': [29, 29, 29, 29, 29],
    'workclass': ['Private', 'Private', 'Private', 'Private', 'Private'],
    'education': ['Prof-school', 'Some-college', 'Doctorate', 'Bachelors', 'Bachelors'],
    'marital_status': ['Married', 'Married', 'Married', 'Married', 'Married'],
    'occupation': ['White-Collar', 'Professional', 'Blue-Collar', 'Other/Unknown', 'Blue-Collar'],
    'race': ['White', 'White', 'White', 'White', 'White'],
    'gender': ['Female', 'Female', 'Male', 'Female', 'Male'],
    'hours_per_week': [38, 38, 38, 38, 38],
    'income': [1, 1, 1, 1, 1]
}

# Create DataFrame
counterfactuals_df = pd.DataFrame(counterfactuals_data)

# Rules to check against
rules = {
    'Education Level': ['Prof-school', 'Some-college', 'Doctorate', 'Bachelors'],
    'Occupation': ['White-Collar', 'Professional', 'Blue-Collar', 'Other/Unknown'],
    'Gender': ['Female', 'Male'],
    'Work Hours': [38],
    'Marital S

After executing this code, the following results were obtained

In [8]:
print(result1)

Traceback (most recent call last):
  File "c:\Users\afred\miniconda3\envs\kdd\lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'education_level'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\afred\REPOS\HI-AI-KDD24-LMM-4-CFs-Explanation\temp_code.py", line 35, in <module>
    rules_count[rule] = counterfactuals_df[column_name].isin(valid_values).sum()
  File "c:\Users\afred\miniconda3\envs\kdd\lib\site-packages\pandas\core\frame.py", line 3807, i

By using this causes, the LLM will produce a final explanation, hopefully using the most important causes

In [9]:
print(explanation)

Based on the data provided and the counterfactual analyses, here are some actionable insights to help you improve your chances of earning more than $50,000 per year:

1. **Education Level**: 
   - **Current Status**: High School Graduate
   - **Recommended Change**: Consider pursuing higher education. Most of the counterfactuals that predicted a higher income had higher education levels such as Professional School, Some College, Bachelors, or Doctorate. This suggests that increasing your level of education could significantly improve your earning potential.

2. **Occupation**:
   - **Current Status**: Blue-Collar
   - **Recommended Change**: Transitioning to a different occupation could be beneficial. Counterfactuals showed that professions such as White-Collar, Professional jobs, and other unknown occupations had a higher likelihood of earning more than $50,000. Exploring opportunities in these fields might help.

3. **Gender**:
   - **Current Status**: Female
   - **Recommendation**:

Now we would like to check the quality of our explanation. As we explain in the paper, we created a close loop evaluation method that checks whether the 

In [10]:
print(code2)

import pandas as pd

# Creating a dataframe with characteristics that are likely to be classified in the positive class
data = {
    'age': [29],
    'workclass': ['Private'],
    'education': ['Bachelors'],  # Changed from HS-grad to Bachelors
    'marital_status': ['Married'],
    'occupation': ['White-Collar'],  # Changed from Blue-Collar to White-Collar
    'race': ['White'],
    'gender': ['Female'],
    'hours_per_week': [38],
    'income': [1]  # Positive class
}

df = pd.DataFrame(data)
df.to_csv('temp_csv.csv', index=False)


After executing the code the LLM correctly generated a counterfactual example.

In [11]:
final_cf

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week
0,29,Private,Bachelors,Married,White-Collar,White,Female,38


We will now try to extract metrics from this counterfactual like the predicted class, rules followed and if it exists in the data set. We ask the LLM for code again and this time it will create a table that we can analyze.

In [12]:
print(code3)

import pandas as pd

# Load the example data
df = pd.read_csv('temp_csv.csv')

# Define the rules and their importance
rules = {
    "Education Level": df['education'].isin(['Bachelors', 'Masters', 'Doctorate', 'Prof-school']).sum(),
    "Occupation": df['occupation'].isin(['Professional', 'Sales', 'Service', 'White-Collar']).sum(),
    "Gender": df['gender'].isin(['Male']).sum(),
    "Work Hours": (df['hours_per_week'] > 40).sum(),
    "Marital Status": df['marital_status'].isin(['Married']).sum(),
    "Race": df['race'].isin(['White']).sum()
}

# Check if the example follows the rules
example = df.iloc[0]
follows_rules = {
    "Education Level": int(example['education'] in ['Bachelors', 'Masters', 'Doctorate', 'Prof-school']),
    "Occupation": int(example['occupation'] in ['Professional', 'Sales', 'Service', 'White-Collar']),
    "Gender": int(example['gender'] == 'Male'),
    "Work Hours": int(example['hours_per_week'] > 40),
    "Marital Status": int(example['marital_status'] == '

In [13]:
eval_df = pd.read_csv('evaluation.csv')
eval_df

FileNotFoundError: [Errno 2] No such file or directory: 'evaluation.csv'

After analyzing the table we get the following results.

In [ ]:
print('Prediction of the generated example: ', prediction)
print('Number of rules generated: ', n_rules)
print('Number of rules followed: ', rules_followed)
print('1st rule followed ', first_rule)
print('2nd rule followed ', second_rule)
print('3rd rule followed ', third_rule)
print('Example exists in data: ', is_in_data)

Prediction of the generated example:  0
Number of rules generated:  5
Number of rules followed:  5
1st rule followed  1
2nd rule followed  1
3rd rule followed  1
Example exists in data:  True
